# Link
https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import numpy as np

import tensorflow as tf
import tf_keras as tfk
import tensorflow_datasets as tfds
import tensorflow_probability as tfp


tfkl = tfk.layers
tfpl = tfp.layers
tfd = tfp.distributions

2024-10-23 13:59:47.106909: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-23 13:59:47.109980: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-23 13:59:47.119387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 13:59:47.135770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 13:59:47.140313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 13:59:47.151936: I tensorflow/core/platform/cpu_feature_gu

In [3]:
print(tf.__version__)  # Should match with tfp.__version__
print(tfp.__version__)

2.17.0
0.24.0


# Load dataset

In [4]:
datasets, datasets_info = tfds.load(name='mnist',
                                    with_info=True,
                                    as_supervised=False)

def _preprocess(sample):
  image = tf.cast(sample['image'], tf.float32) / 255.  # Scale to unit interval.
  image = image < tf.random.uniform(tf.shape(image))   # Randomly binarize.
  return image, image

train_dataset = (datasets['train']
                 .map(_preprocess)
                 .batch(256)
                 .prefetch(tf.data.AUTOTUNE)
                 .shuffle(int(10e3)))
eval_dataset = (datasets['test']
                .map(_preprocess)
                .batch(256)
                .prefetch(tf.data.AUTOTUNE))

# VAE

In [5]:
encoded_size = 16
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)

In [6]:
input_shape = (28, 28, 1)
base_depth = 32

In [7]:
encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    tfkl.Conv2D(base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=1,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(2 * base_depth, 5, strides=2,
                padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(4 * encoded_size, 7, strides=1,
                padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Flatten(),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
               activation=None),
    tfpl.MultivariateNormalTriL(
        encoded_size,
        activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
])

In [8]:
decoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=[encoded_size]),
    tfkl.Reshape([1, 1, encoded_size]),
    tfkl.Conv2DTranspose(2 * base_depth, 7, strides=1,
                         padding='valid', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(2 * base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=2,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2DTranspose(base_depth, 5, strides=1,
                         padding='same', activation=tf.nn.leaky_relu),
    tfkl.Conv2D(filters=1, kernel_size=5, strides=1,
                padding='same', activation=None),
    tfkl.Flatten(),
    tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits),
])

In [9]:
vae = tfk.Model(inputs=encoder.inputs,
                outputs=decoder(encoder.outputs[0]))

In [11]:
negloglik = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tfk.optimizers.Adam(learning_rate=1e-3),
            loss=negloglik)

_ = vae.fit(train_dataset,
            epochs=15,
            validation_data=eval_dataset)

Epoch 1/15
 64/235 [=======>......................] - ETA: 6:14 - loss: 275.3081

KeyboardInterrupt: 

Here received another odd error; though at least the packages are now loading!

In [12]:
type(encoder.inputs)

list

In [13]:
encoder.inputs

[<KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'input_1')>]

In [14]:
encoder.outputs

In [15]:
vae.outputs

In [16]:
# We'll just examine ten random digits.
x = next(iter(eval_dataset))[0][:10]
xhat = vae(x)
assert isinstance(xhat, tfd.Distribution)

2024-10-23 14:42:05.532316: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [36]:
xhat

<tfp.distributions._TensorCoercible 'tensor_coercible' batch_shape=[10] event_shape=[28, 28, 1] dtype=float32>

In [29]:
zhat = encoder(x)

print(zhat)
print(zhat[0])

tfp.distributions._TensorCoercible("tensor_coercible", batch_shape=[10], event_shape=[16], dtype=float32)
tf.Tensor(
[-0.52426577  0.37401426  0.29246086 -0.06266586  0.79769635 -0.9415609
 -1.2085022   0.5771363   0.15888679 -0.9138141  -1.6856306   0.19799906
 -0.8239684   1.0100198  -0.6871591  -0.5799698 ], shape=(16,), dtype=float32)


In [35]:
zhat[9]

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ 0.1530599 ,  1.7239944 ,  0.8953391 , -0.6645279 ,  0.4168939 ,
        1.3389987 ,  1.2228439 , -0.5738822 , -1.2628341 ,  1.3617172 ,
       -0.6584424 ,  0.07537699, -0.22646374, -0.7286111 , -1.3891656 ,
        1.457211  ], dtype=float32)>

Thoughts:
- could write a stackoverflow
- but it is not really the point of what I am trying to do  
- be better to follow the other tutorial with more explicit loss functions and sampling setups  
- indeed, the aim was to get something working in jax and flax, not to mess around with some arcane functionality of tfpl + keras

Update 2024-10-26: think have figured it out!

Yes, tensor coercion obtains a single sample, as following code from github shows:
```
def __init__(self,
               make_distribution_fn,
               convert_to_tensor_fn=tfd.Distribution.sample,
               **kwargs):
```
from DistributionLambda class at https://github.com/tensorflow/probability/blob/main/tensorflow_probability/python/layers/distribution_layer.py#L96